# HOWTO - Use the Vantage APIs to create a Workflow

## Overview
This example notebook will show how to use the output product of one service as the input of another one on Vantage using the platform's APIs.

Steps:

1. Retrieve service's details
2. Retrieve collection's details
3. Run the jobs and get the output products
4. Run the second service using the output files of the previous jobs
5. Download the final product

More information about the available APIs can be found in the [Hal Browser](https://vantage.earthi.world/secure/api/v2.0/).

## 0. Preparation steps

In [1]:
import requests
import json

In [2]:
import getpass

user = getpass.getpass('Username: ')

Username:  ·········


In [3]:
api = getpass.getpass('API Key: ')

API Key:  ································································


In [4]:
endpoint = 'https://vantage.earthi.world/secure/api/v2.0'

## 1. Retrieve service's details

#### 1.1 Filter the services

In [86]:
query = { 'sort':'type,name','filter':'forest'}
response = requests.get(endpoint + '/services/search/findByFilterOnly', auth=(user,api), params=(query))
response.status_code

200

In [87]:
json_data = json.loads(response.text)
for service in json_data['_embedded']['services']:
    print("Name: {0}\nDescription: {1}\nID: {2}\n".format(service['name'], service['description'], service['id']))

Name: ForestCover
Description: Produces a forest cover map for the first video frame.
ID: 20



#### 1.2 Show service detailed information

In [88]:
service_id = '20'

In [89]:
response = requests.get(endpoint + '/services/' + service_id, auth=(user,api))
response.status_code

200

In [90]:
response.json()

{'id': 20,
 'name': 'ForestCover',
 'description': 'Produces a forest cover map for the first video frame.',
 'type': 'PROCESSOR',
 'dockerTag': 'eovep/forestcover',
 'stripProxyPath': True,
 'licence': 'OPEN',
 'status': 'AVAILABLE',
 'serviceDescriptor': {'id': 'ForestCover',
  'title': 'ForestCover',
  'description': 'Produces a forest cover map for the first video frame.',
  'version': '0.1',
  'storeSupported': False,
  'statusSupported': False,
  'serviceType': 'Java',
  'serviceProvider': 'ForestCover',
  'dataInputs': [{'id': 'VideoFile',
    'title': 'Video File',
    'description': 'Input video file to process',
    'minOccurs': 0,
    'maxOccurs': 0,
    'data': 'LITERAL',
    'defaultAttrs': {'dataType': 'string'},
    'supportedAttrs': None}],
  'dataOutputs': [{'id': 'output',
    'title': 'output',
    'description': 'Output cloud mask',
    'minOccurs': 0,
    'maxOccurs': 0,
    'data': 'LITERAL',
    'defaultAttrs': {'dataType': 'string'},
    'supportedAttrs': None,


## 2. Retrieve collection's details

#### 2.1 Filtering the collections

In [91]:
query = { 'sort':'name', 'filter':'vivid'}
response = requests.get(endpoint + '/collections/search/parametricFind', auth=(user,api), params=(query))
response.status_code

200

In [92]:
json_data = json.loads(response.text)

for collection in json_data['_embedded']['collections']:
    print("Name: {0}\nDescription: {1}\nID: {2}".format(collection['name'], collection['description'], collection['identifier']))

Name: Vivid-X2
Description: video collection
ID: eovep2e8134bcdaf74f528b23b6dba883bcde


#### 2.2 Get the products of a specific collection

In [93]:
collection_id = 'eovep2e8134bcdaf74f528b23b6dba883bcde'

In [94]:
response = requests.get(endpoint + '/search?catalogue=REMOTE_DATA_commercialData&geometry=&remoteDataCollection_commercialData=' + collection_id, auth=(user,api))
response.status_code

200

In [95]:
json_data = json.loads(response.text)

for feature in json_data['features']:
    print("{0} URL: {1}".format(feature['properties']['productIdentifier'], feature['properties']['platformUrl']))

VX020003b9 URL: https://esrin-data-input.s3.eu-west-2.amazonaws.com/Vivid-X2/VX020003b9.zip
VX020003b7 URL: https://esrin-data-input.s3.eu-west-2.amazonaws.com/Vivid-X2/VX020003b7.zip
VX020003b5 URL: https://esrin-data-input.s3.eu-west-2.amazonaws.com/Vivid-X2/VX020003b5.zip
VX020003b4 URL: https://esrin-data-input.s3.eu-west-2.amazonaws.com/Vivid-X2/VX020003b4.zip
VX020003b2 URL: https://esrin-data-input.s3.eu-west-2.amazonaws.com/Vivid-X2/VX020003b2.zip
VX020003b6 URL: https://esrin-data-input.s3.eu-west-2.amazonaws.com/Vivid-X2/VX020003b6.zip
VX020003b0 URL: https://esrin-data-input.s3.eu-west-2.amazonaws.com/Vivid-X2/VX020003b0.zip
VX020003af URL: https://esrin-data-input.s3.eu-west-2.amazonaws.com/Vivid-X2/VX020003af.zip
VX020003ae URL: https://esrin-data-input.s3.eu-west-2.amazonaws.com/Vivid-X2/VX020003ae.zip
VX020003ad URL: https://esrin-data-input.s3.eu-west-2.amazonaws.com/Vivid-X2/VX020003ad.zip
VX020003a8 URL: https://esrin-data-input.s3.eu-west-2.amazonaws.com/Vivid-X2/VX0

## 3.Run the jobs and get the output products

#### 3.1 Prepare the job

There are 2 completed jobs already on the platform we can take as an example.

JOB 962 with input file: https://esrin-data-input.s3.eu-west-2.amazonaws.com/Vivid-X2/VX020003ad.zip


JOB 963 with input file: https://esrin-data-input.s3.eu-west-2.amazonaws.com/Vivid-X2/VX02000186.zip

In [5]:
headers={'Accept':'application/hal+json', 'Content-Type':'application/hal+json'}
raw_data='{"service":"https://vantage.earthi.world/secure/api/v2.0/services/20","inputs":{"VideoFile":["https://esrin-data-input.s3.eu-west-2.amazonaws.com/Vivid-X2/VX020003ad.zip"]},"label":"forestMask1","parent":null}'
# Comment the following to avoid launching a job 
response = requests.post(endpoint + '/jobConfigs/', auth=(user,api), data=raw_data,headers=headers)
response.status_code

201

In [6]:
response.json()

{'id': 517,
 'inputs': {'VideoFile': ['https://esrin-data-input.s3.eu-west-2.amazonaws.com/Vivid-X2/VX020003ad.zip']},
 'label': 'forestMask1',
 'systematicParameter': None,
 'tileBasedJobOrdering': None,
 '_embedded': {'owner': {'role': 'CONTENT_AUTHORITY',
   'name': 'demo.jhub',
   'id': 50,
   '_links': {'self': {'href': 'https://vantage.earthi.world/secure/api/v2.0/users/50{?projection}',
     'templated': True}}},
  'service': {'owner': {'role': 'ADMIN',
    'name': 'xenofon.karagiannis',
    'id': 24,
    '_links': {'self': {'href': 'https://vantage.earthi.world/secure/api/v2.0/users/24{?projection}',
      'templated': True}}},
   'description': 'Produces a forest cover map for the first video frame.',
   'status': 'AVAILABLE',
   'costingExpression': {'type': None,
    'associatedId': None,
    'costExpression': '1',
    'estimatedCostExpression': None},
   'dockerTag': 'eovep/forestcover',
   'access': {'published': True,
    'publishRequested': False,
    'currentLevel': 'SU

We get the job's configuration ID 

#### 3.2 Get the estimated cost for the job

In [7]:
config_id_1 = '517'

In [8]:
response = requests.get(endpoint + '/estimateCost' + '/jobConfig/' + config_id_1, auth=(user,api))
response.status_code

200

In [9]:
response.json()

{'estimatedCost': 1, 'recurrence': 'HOURLY', 'currentWalletBalance': 117}

#### 3.3 Launch the job

In [10]:
config_id1 = '517'

In [11]:
# Commented to avoid launching a job 
response = requests.post(endpoint + '/jobConfigs/' + config_id_1 + '/launch', auth=(user,api))
response.status_code

202

In [12]:
# Commented to avoid launching a job 
response.json()

{'content': {'id': 979,
  'extId': '4f4b8e62-6377-4ffd-9304-5d510e545908',
  'startTime': None,
  'endTime': None,
  'status': 'PENDING',
  'phase': 'CREATED',
  'workerId': None,
  'stage': None,
  'guiUrl': None,
  'guiEndpoint': None,
  'queuePosition': -1,
  'outputs': None,
  'parent': False,
  'costQuotation': {'cost': 1, 'recurrence': 'HOURLY'},
  'inputTile': None,
  'inputDate': None},
 '_links': {'self': {'href': 'https://vantage.earthi.world/secure/api/v2.0/jobs/979'},
  'job': {'href': 'https://vantage.earthi.world/secure/api/v2.0/jobs/979{?projection}',
   'templated': True},
  'logs': {'href': 'https://vantage.earthi.world/secure/api/v2.0/jobs/979/logs'},
  'cancel': {'href': 'https://vantage.earthi.world/secure/api/v2.0/jobs/979/cancel'}}}

The response shows the Job's ID

In [13]:
job_id_1='979'

#### 3.4 Launch the second job

In [14]:
headers={'Accept':'application/hal+json', 'Content-Type':'application/hal+json'}
raw_data='{"service":"https://vantage.earthi.world/secure/api/v2.0/services/20","inputs":{"VideoFile":["https://esrin-data-input.s3.eu-west-2.amazonaws.com/Vivid-X2/VX02000186.zip"]},"label":"forestMask2","parent":null}'
# Comment the following to avoid launching a job 
response = requests.post(endpoint + '/jobConfigs/', auth=(user,api), data=raw_data,headers=headers)
response.status_code


201

In [15]:
response.json()

{'id': 518,
 'inputs': {'VideoFile': ['https://esrin-data-input.s3.eu-west-2.amazonaws.com/Vivid-X2/VX02000186.zip']},
 'label': 'forestMask2',
 'systematicParameter': None,
 'tileBasedJobOrdering': None,
 '_embedded': {'owner': {'role': 'CONTENT_AUTHORITY',
   'name': 'demo.jhub',
   'id': 50,
   '_links': {'self': {'href': 'https://vantage.earthi.world/secure/api/v2.0/users/50{?projection}',
     'templated': True}}},
  'service': {'owner': {'role': 'ADMIN',
    'name': 'xenofon.karagiannis',
    'id': 24,
    '_links': {'self': {'href': 'https://vantage.earthi.world/secure/api/v2.0/users/24{?projection}',
      'templated': True}}},
   'description': 'Produces a forest cover map for the first video frame.',
   'status': 'AVAILABLE',
   'costingExpression': {'type': None,
    'associatedId': None,
    'costExpression': '1',
    'estimatedCostExpression': None},
   'dockerTag': 'eovep/forestcover',
   'access': {'published': True,
    'publishRequested': False,
    'currentLevel': 'SU

In [16]:
config_id_2 = '518'

In [17]:
response = requests.get(endpoint + '/estimateCost' + '/jobConfig/' + config_id_2, auth=(user,api))
response.status_code

200

In [18]:
response.json()

{'estimatedCost': 1, 'recurrence': 'HOURLY', 'currentWalletBalance': 116}

In [19]:
# Commented to avoid launching a job 
response = requests.post(endpoint + '/jobConfigs/' + config_id_2 + '/launch', auth=(user,api))
response.status_code

202

In [20]:
# Commented to avoid launching a job 
response.json()

{'content': {'id': 980,
  'extId': '5f2fd9cb-a034-46a8-816d-501388f0177c',
  'startTime': None,
  'endTime': None,
  'status': 'PENDING',
  'phase': 'CREATED',
  'workerId': None,
  'stage': None,
  'guiUrl': None,
  'guiEndpoint': None,
  'queuePosition': -1,
  'outputs': None,
  'parent': False,
  'costQuotation': {'cost': 1, 'recurrence': 'HOURLY'},
  'inputTile': None,
  'inputDate': None},
 '_links': {'self': {'href': 'https://vantage.earthi.world/secure/api/v2.0/jobs/980'},
  'job': {'href': 'https://vantage.earthi.world/secure/api/v2.0/jobs/980{?projection}',
   'templated': True},
  'logs': {'href': 'https://vantage.earthi.world/secure/api/v2.0/jobs/980/logs'},
  'cancel': {'href': 'https://vantage.earthi.world/secure/api/v2.0/jobs/980/cancel'}}}

In [21]:
job_id_2='980'

#### 3.5 Filter the jobs by user's ID

In [100]:
query = { 'sort':'id,DESC','owner':'https://vantage.earthi.world/secure/api/v2.0/users/50'}
response = requests.get(endpoint + '/jobs/search/parametricFind', auth=(user,api), params=(query))
response.status_code

200

In [101]:
response.json()

{'_embedded': {'jobs': [{'owner': {'role': 'CONTENT_AUTHORITY',
     'name': 'demo.jhub',
     'id': 50,
     '_links': {'self': {'href': 'https://vantage.earthi.world/secure/api/v2.0/users/50{?projection}',
       'templated': True}}},
    'serviceName': 'ForestCover',
    'endTime': '2021-07-16T14:20:35.432',
    'startTime': '2021-07-16T14:20:08.139',
    'guiUrl': None,
    'queuePosition': -1,
    'parent': False,
    'status': 'ERROR',
    'extId': '553e9944-c152-4242-9052-150e9cb94c1e',
    'access': {'published': False,
     'publishRequested': False,
     'currentLevel': 'SUPERUSER'},
    'systematicParameter': None,
    'stage': 'Step 2 of 3: Processing',
    'id': 969,
    '_links': {'self': {'href': 'https://vantage.earthi.world/secure/api/v2.0/jobs/969'},
     'job': {'href': 'https://vantage.earthi.world/secure/api/v2.0/jobs/969{?projection}',
      'templated': True},
     'logs': {'href': 'https://vantage.earthi.world/secure/api/v2.0/jobs/969/logs'},
     'subJobs': {'h

#### 3.6 Get the jobs information

Reference jobs are 979 and 980

In [22]:
job_id_1='979'

In [103]:
response = requests.get(endpoint + '/jobs/' + job_id_1, auth=(user,api))
response.status_code

200

In [104]:
response.json()

{'id': 962,
 'extId': '2aabb4d6-144c-404f-b67b-d8886c943d32',
 'startTime': '2021-07-16T09:39:40.877',
 'endTime': '2021-07-16T09:58:03.788',
 'status': 'COMPLETED',
 'phase': 'OUTPUT_LIST',
 'workerId': 'LOCAL1',
 'stage': 'Step 3 of 3: Output-List',
 'guiUrl': None,
 'guiEndpoint': None,
 'queuePosition': -1,
 'outputs': {'output': ['eovep://outputProduct/2aabb4d6-144c-404f-b67b-d8886c943d32/output/VX020003ad.tif']},
 'parent': False,
 'costQuotation': {'cost': 1, 'recurrence': 'HOURLY'},
 'inputTile': None,
 'inputDate': None,
 '_embedded': {'owner': {'role': 'CONTENT_AUTHORITY',
   'name': 'demo.jhub',
   'id': 50,
   '_links': {'self': {'href': 'https://vantage.earthi.world/secure/api/v2.0/users/50{?projection}',
     'templated': True}}},
  'outputFiles': [{'owner': {'role': 'CONTENT_AUTHORITY',
     'name': 'demo.jhub',
     'id': 50,
     '_links': {'self': {'href': 'https://vantage.earthi.world/secure/api/v2.0/users/50{?projection}',
       'templated': True}}},
    'restoId':

Get the second job information

In [23]:
job_id_2='980'

In [106]:
response = requests.get(endpoint + '/jobs/' + job_id_2, auth=(user,api))
response.status_code

200

In [107]:
response.json()

{'id': 963,
 'extId': 'c57c503a-1b43-4fe9-bece-845840eab0fd',
 'startTime': '2021-07-16T09:41:56.26',
 'endTime': '2021-07-16T09:58:44.889',
 'status': 'COMPLETED',
 'phase': 'OUTPUT_LIST',
 'workerId': 'LOCAL1',
 'stage': 'Step 3 of 3: Output-List',
 'guiUrl': None,
 'guiEndpoint': None,
 'queuePosition': -1,
 'outputs': {'output': ['eovep://outputProduct/c57c503a-1b43-4fe9-bece-845840eab0fd/output/VX02000186.tif']},
 'parent': False,
 'costQuotation': {'cost': 1, 'recurrence': 'HOURLY'},
 'inputTile': None,
 'inputDate': None,
 '_embedded': {'owner': {'role': 'CONTENT_AUTHORITY',
   'name': 'demo.jhub',
   'id': 50,
   '_links': {'self': {'href': 'https://vantage.earthi.world/secure/api/v2.0/users/50{?projection}',
     'templated': True}}},
  'outputFiles': [{'owner': {'role': 'CONTENT_AUTHORITY',
     'name': 'demo.jhub',
     'id': 50,
     '_links': {'self': {'href': 'https://vantage.earthi.world/secure/api/v2.0/users/50{?projection}',
       'templated': True}}},
    'restoId': 

#### 3.7 [OPTIONAL]  Retrieve the logs from the jobs

In [24]:
job_id_1='979'
job_id_2='980'


In [109]:
response = requests.get(endpoint + '/jobs/' + job_id_1 + '/logs', auth=(user,api))
response.status_code

200

In [110]:
response.json()

[{'timestamp': '2021-07-16T09:41:48.687953795+00:00',
  'message': '+++ readlink -f /home/worker/processor/workflow.sh\n'},
 {'timestamp': '2021-07-16T09:41:48.690127974+00:00',
  'message': '++ dirname /home/worker/processor/workflow.sh\n'},
 {'timestamp': '2021-07-16T09:41:48.692116461+00:00',
  'message': '+ WORKFLOW=/home/worker/processor\n'},
 {'timestamp': '2021-07-16T09:41:48.692131016+00:00',
  'message': '+ WORKER_DIR=/home/worker\n'},
 {'timestamp': '2021-07-16T09:41:48.692135446+00:00',
  'message': '+ IN_DIR=/home/worker/workDir/inDir\n'},
 {'timestamp': '2021-07-16T09:41:48.692139253+00:00',
  'message': '+ OUT_DIR=/home/worker/workDir/outDir\n'},
 {'timestamp': '2021-07-16T09:41:48.692142757+00:00',
  'message': '+ WPS_PROPS=/home/worker/workDir/WPS-INPUT.properties\n'},
 {'timestamp': '2021-07-16T09:41:48.692146305+00:00',
  'message': '+ PROC_DIR=/home/worker/procDir\n'},
 {'timestamp': '2021-07-16T09:41:48.692505870+00:00',
  'message': '++ date --utc +%Y%m%d_%H%M%SZ\n

And for the second job

In [ ]:
response = requests.get(endpoint + '/jobs/' + job_id_2 + '/logs', auth=(user,api))
response.status_code

In [ ]:
response.json()

#### 3.8 Get the list of the job's output files

In [25]:
job_id_1='979'
job_id_2='980'

In [26]:
response = requests.get(endpoint + '/jobs/' + job_id_1 + '/outputFiles?projection=detailedPlatformFile', auth=(user,api))
response.status_code

200

Get the URI of the product

In [27]:
response.json()

{'_embedded': {'platformFiles': [{'owner': {'role': 'CONTENT_AUTHORITY',
     'name': 'demo.jhub',
     'id': 50,
     '_links': {'self': {'href': 'https://vantage.earthi.world/secure/api/v2.0/users/50{?projection}',
       'templated': True}}},
    'restoId': '54ce3b04-5dc3-5e4d-9e5b-f9edaf2fb5dc',
    'filename': '4f4b8e62-6377-4ffd-9304-5d510e545908/output/output.zip',
    'metadata': {'type': 'Feature',
     'properties': {'collection': 'eovepOutputs',
      'license': {'licenseId': 'unlicensed',
       'hasToBeSigned': 'never',
       'grantedCountries': None,
       'grantedOrganizationCountries': None,
       'grantedFlags': None,
       'viewService': 'public',
       'signatureQuota': -1,
       'description': {'shortName': 'No license'}},
      'productIdentifier': '4f4b8e62-6377-4ffd-9304-5d510e545908_output/output.zip',
      'parentIdentifier': None,
      'title': '4f4b8e62-6377-4ffd-9304-5d510e545908_output/output.zip',
      'description': None,
      'organisationName'

URI Job1 : eovep://outputProduct/4f4b8e62-6377-4ffd-9304-5d510e545908/output/output.zip

And for the second job

In [30]:
response = requests.get(endpoint + '/jobs/' + job_id_2 + '/outputFiles?projection=detailedPlatformFile', auth=(user,api))
response.status_code

200

In [31]:
response.json()

{'_embedded': {'platformFiles': [{'owner': {'role': 'CONTENT_AUTHORITY',
     'name': 'demo.jhub',
     'id': 50,
     '_links': {'self': {'href': 'https://vantage.earthi.world/secure/api/v2.0/users/50{?projection}',
       'templated': True}}},
    'restoId': '2f993756-5e09-5bf0-a731-49f12eff4735',
    'filename': '5f2fd9cb-a034-46a8-816d-501388f0177c/output/output.zip',
    'metadata': {'type': 'Feature',
     'properties': {'collection': 'eovepOutputs',
      'license': {'licenseId': 'unlicensed',
       'hasToBeSigned': 'never',
       'grantedCountries': None,
       'grantedOrganizationCountries': None,
       'grantedFlags': None,
       'viewService': 'public',
       'signatureQuota': -1,
       'description': {'shortName': 'No license'}},
      'productIdentifier': '5f2fd9cb-a034-46a8-816d-501388f0177c_output/output.zip',
      'parentIdentifier': None,
      'title': '5f2fd9cb-a034-46a8-816d-501388f0177c_output/output.zip',
      'description': None,
      'organisationName'

URI Job2 : eovep://outputProduct/5f2fd9cb-a034-46a8-816d-501388f0177c/output/output.zip

#### 3.9 Show the details of one product

In [32]:
job_id_1='979'
job_id_2='980'
out_1='2498'
out_2='2499'

In [33]:
response = requests.get(endpoint + '/jobs/' + job_id_1 + '/outputFiles/' + out_1, auth=(user,api))
response.status_code

200

In [34]:
response.json()

{'owner': {'role': 'CONTENT_AUTHORITY',
  'name': 'demo.jhub',
  'id': 50,
  '_links': {'self': {'href': 'https://vantage.earthi.world/secure/api/v2.0/users/50{?projection}',
    'templated': True}}},
 'restoId': '54ce3b04-5dc3-5e4d-9e5b-f9edaf2fb5dc',
 'filename': '4f4b8e62-6377-4ffd-9304-5d510e545908/output/output.zip',
 'access': {'published': False,
  'publishRequested': False,
  'currentLevel': 'SUPERUSER'},
 'type': 'OUTPUT_PRODUCT',
 'id': 2498,
 '_links': {'self': {'href': 'https://vantage.earthi.world/secure/api/v2.0/platformFiles/2498'},
  'file': {'href': 'https://vantage.earthi.world/secure/api/v2.0/platformFiles/2498{?projection}',
   'templated': True},
  'download': {'href': 'https://vantage.earthi.world/secure/api/v2.0/platformFiles/2498/dl'},
  'owner': {'href': 'https://vantage.earthi.world/secure/api/v2.0/platformFiles/2498/owner'},
  'jobs': {'href': 'https://vantage.earthi.world/secure/api/v2.0/platformFiles/2498/jobs'},
  'collection': {'href': 'https://vantage.ea

And for the second output

In [35]:
response = requests.get(endpoint + '/jobs/' + job_id_2 + '/outputFiles/' + out_2, auth=(user,api))
response.status_code

200

In [36]:
response.json()

{'owner': {'role': 'CONTENT_AUTHORITY',
  'name': 'demo.jhub',
  'id': 50,
  '_links': {'self': {'href': 'https://vantage.earthi.world/secure/api/v2.0/users/50{?projection}',
    'templated': True}}},
 'restoId': '2f993756-5e09-5bf0-a731-49f12eff4735',
 'filename': '5f2fd9cb-a034-46a8-816d-501388f0177c/output/output.zip',
 'access': {'published': False,
  'publishRequested': False,
  'currentLevel': 'SUPERUSER'},
 'type': 'OUTPUT_PRODUCT',
 'id': 2499,
 '_links': {'self': {'href': 'https://vantage.earthi.world/secure/api/v2.0/platformFiles/2499'},
  'file': {'href': 'https://vantage.earthi.world/secure/api/v2.0/platformFiles/2499{?projection}',
   'templated': True},
  'download': {'href': 'https://vantage.earthi.world/secure/api/v2.0/platformFiles/2499/dl'},
  'owner': {'href': 'https://vantage.earthi.world/secure/api/v2.0/platformFiles/2499/owner'},
  'jobs': {'href': 'https://vantage.earthi.world/secure/api/v2.0/platformFiles/2499/jobs'},
  'collection': {'href': 'https://vantage.ea

## 4. Run the second service using the output files of the previous jobs

#### 4.1 Get the second service information

In [119]:
query = { 'sort':'type,name','filter':'symmetric'}
response = requests.get(endpoint + '/services/search/findByFilterOnly', auth=(user,api), params=(query))
response.status_code

200

In [120]:
json_data = json.loads(response.text)
for service in json_data['_embedded']['services']:
    print("Name: {0}\nDescription: {1}\nID: {2}\n".format(service['name'], service['description'], service['id']))

Name: SymmetricDifference
Description: Calculates the difference (change) between two (georeferenced) binary masks (values 0,1)
ID: 21



In [37]:
service_id='21'

In [38]:
response = requests.get(endpoint + '/services/' + service_id, auth=(user,api))
response.status_code

200

In [39]:
response.json()

{'id': 21,
 'name': 'SymmetricDifference',
 'description': 'Calculates the difference (change) between two (georeferenced) binary masks (values 0,1)',
 'type': 'PROCESSOR',
 'dockerTag': 'eovep/symmetricdifference',
 'stripProxyPath': True,
 'licence': 'OPEN',
 'status': 'AVAILABLE',
 'serviceDescriptor': {'id': 'SymmetricDifference',
  'title': 'SymmetricDifference',
  'description': 'Calculates the difference (change) between two (georeferenced) binary masks (values 0,1)',
  'version': '0.1',
  'storeSupported': False,
  'statusSupported': False,
  'serviceType': 'Java',
  'serviceProvider': 'SymmetricDifference',
  'dataInputs': [{'id': 'ref_mask_path',
    'title': 'ref_mask_path',
    'description': 'Reference binary mask (geoTiff)',
    'minOccurs': 0,
    'maxOccurs': 0,
    'data': 'LITERAL',
    'defaultAttrs': {'dataType': 'string'},
    'supportedAttrs': None},
   {'id': 'target_mask_path',
    'title': 'target_mask_path',
    'description': 'Target binary mask (geoTiff)',
 

#### 4.2 Prepare the second service

URI output file of first job:
eovep://outputProduct/4f4b8e62-6377-4ffd-9304-5d510e545908/output/output.zip

URI output file of second job:
eovep://outputProduct/5f2fd9cb-a034-46a8-816d-501388f0177c/output/output.zip

In [40]:
headers={'Accept':'application/hal+json', 'Content-Type':'application/hal+json'}
raw_data='{"service":"https://vantage.earthi.world/secure/api/v2.0/services/21","inputs":{"ref_mask_path":["eovep://outputProduct/4f4b8e62-6377-4ffd-9304-5d510e545908/output/output.zip"],"target_mask_path":["eovep://outputProduct/5f2fd9cb-a034-46a8-816d-501388f0177c/output/output.zip"]},"label":"SymmetricDifferenceWorkflow","parent":null}'
# Comment the following to avoid launching a job 
response = requests.post(endpoint + '/jobConfigs/', auth=(user,api), data=raw_data,headers=headers)
response.status_code



201

In [41]:
response.json()

{'id': 527,
 'inputs': {'ref_mask_path': ['eovep://outputProduct/4f4b8e62-6377-4ffd-9304-5d510e545908/output/output.zip'],
  'target_mask_path': ['eovep://outputProduct/5f2fd9cb-a034-46a8-816d-501388f0177c/output/output.zip']},
 'label': 'SymmetricDifferenceWorkflow',
 'systematicParameter': None,
 'tileBasedJobOrdering': None,
 '_embedded': {'owner': {'role': 'CONTENT_AUTHORITY',
   'name': 'demo.jhub',
   'id': 50,
   '_links': {'self': {'href': 'https://vantage.earthi.world/secure/api/v2.0/users/50{?projection}',
     'templated': True}}},
  'service': {'owner': {'role': 'ADMIN',
    'name': 'xenofon.karagiannis',
    'id': 24,
    '_links': {'self': {'href': 'https://vantage.earthi.world/secure/api/v2.0/users/24{?projection}',
      'templated': True}}},
   'description': 'Calculates the difference (change) between two (georeferenced) binary masks (values 0,1)',
   'status': 'AVAILABLE',
   'costingExpression': {'type': None,
    'associatedId': None,
    'costExpression': '1',
   

Get the job's configuration ID

#### 4.3 Get the estimated cost for the job

In [124]:
config_id = '520'

In [125]:
response = requests.get(endpoint + '/estimateCost' + '/jobConfig/' + config_id, auth=(user,api))
response.status_code

200

In [126]:
response.json()

{'estimatedCost': 1, 'recurrence': 'HOURLY', 'currentWalletBalance': 18}

#### 4.4 Launch the job

There is one job completed on the platform already which can be taken as an example

JOB: 966

In [42]:
config_id = '527'

In [43]:
# Commented to avoid launching a job 
response = requests.post(endpoint + '/jobConfigs/' + config_id + '/launch', auth=(user,api))
response.status_code

202

In [44]:
response.json()

{'content': {'id': 982,
  'extId': '74f3eff7-1c4c-41da-af71-ff1255b21650',
  'startTime': None,
  'endTime': None,
  'status': 'PENDING',
  'phase': 'CREATED',
  'workerId': None,
  'stage': None,
  'guiUrl': None,
  'guiEndpoint': None,
  'queuePosition': -1,
  'outputs': None,
  'parent': False,
  'costQuotation': {'cost': 1, 'recurrence': 'HOURLY'},
  'inputTile': None,
  'inputDate': None},
 '_links': {'self': {'href': 'https://vantage.earthi.world/secure/api/v2.0/jobs/982'},
  'job': {'href': 'https://vantage.earthi.world/secure/api/v2.0/jobs/982{?projection}',
   'templated': True},
  'logs': {'href': 'https://vantage.earthi.world/secure/api/v2.0/jobs/982/logs'},
  'cancel': {'href': 'https://vantage.earthi.world/secure/api/v2.0/jobs/982/cancel'}}}

#### 4.5 Get the job logs

In [45]:
job_id='982'

In [46]:
response = requests.get(endpoint + '/jobs/' + job_id + '/logs', auth=(user,api))
response.status_code

200

In [47]:
response.json()

[{'timestamp': '2021-07-19T09:01:18.635535789+00:00',
  'message': '+++ readlink -f /home/worker/processor/workflow.sh\n'},
 {'timestamp': '2021-07-19T09:01:18.636364485+00:00',
  'message': '++ dirname /home/worker/processor/workflow.sh\n'},
 {'timestamp': '2021-07-19T09:01:18.638163634+00:00',
  'message': '+ WORKFLOW=/home/worker/processor\n'},
 {'timestamp': '2021-07-19T09:01:18.638536353+00:00',
  'message': '+ WORKER_DIR=/home/worker\n'},
 {'timestamp': '2021-07-19T09:01:18.638547273+00:00',
  'message': '+ IN_DIR=/home/worker/workDir/inDir\n'},
 {'timestamp': '2021-07-19T09:01:18.638551135+00:00',
  'message': '+ OUT_DIR=/home/worker/workDir/outDir\n'},
 {'timestamp': '2021-07-19T09:01:18.638554847+00:00',
  'message': '+ WPS_PROPS=/home/worker/workDir/WPS-INPUT.properties\n'},
 {'timestamp': '2021-07-19T09:01:18.638557841+00:00',
  'message': '+ PROC_DIR=/home/worker/procDir\n'},
 {'timestamp': '2021-07-19T09:01:18.638799140+00:00',
  'message': '++ date --utc +%Y%m%d_%H%M%SZ\n

## 5. Download the final product

#### 5.1 Get the list of the job's output files

In [48]:
job_id='982'

In [49]:
response = requests.get(endpoint + '/jobs/' + job_id + '/outputFiles', auth=(user,api))
response.status_code

200

In [50]:
response.json()

{'_embedded': {'platformFiles': [{'owner': {'role': 'CONTENT_AUTHORITY',
     'name': 'demo.jhub',
     'id': 50,
     '_links': {'self': {'href': 'https://vantage.earthi.world/secure/api/v2.0/users/50{?projection}',
       'templated': True}}},
    'restoId': 'e0453096-32ad-5f24-9953-2247aa48c95c',
    'filename': '74f3eff7-1c4c-41da-af71-ff1255b21650/output/output.zip',
    'access': {'published': False,
     'publishRequested': False,
     'currentLevel': 'SUPERUSER'},
    'type': 'OUTPUT_PRODUCT',
    'id': 2501,
    '_links': {'self': {'href': 'https://vantage.earthi.world/secure/api/v2.0/platformFiles/2501'},
     'file': {'href': 'https://vantage.earthi.world/secure/api/v2.0/platformFiles/2501{?projection}',
      'templated': True},
     'download': {'href': 'https://vantage.earthi.world/secure/api/v2.0/platformFiles/2501/dl'},
     'owner': {'href': 'https://vantage.earthi.world/secure/api/v2.0/platformFiles/2501/owner'},
     'jobs': {'href': 'https://vantage.earthi.world/sec

#### 5.2 Show the details of one product

In [51]:
output_file='2501'
response = requests.get(endpoint + '/jobs/' + job_id + '/outputFiles/' + output_file, auth=(user,api))
response.status_code

200

In [52]:
response.json()

{'owner': {'role': 'CONTENT_AUTHORITY',
  'name': 'demo.jhub',
  'id': 50,
  '_links': {'self': {'href': 'https://vantage.earthi.world/secure/api/v2.0/users/50{?projection}',
    'templated': True}}},
 'restoId': 'e0453096-32ad-5f24-9953-2247aa48c95c',
 'filename': '74f3eff7-1c4c-41da-af71-ff1255b21650/output/output.zip',
 'access': {'published': False,
  'publishRequested': False,
  'currentLevel': 'SUPERUSER'},
 'type': 'OUTPUT_PRODUCT',
 'id': 2501,
 '_links': {'self': {'href': 'https://vantage.earthi.world/secure/api/v2.0/platformFiles/2501'},
  'file': {'href': 'https://vantage.earthi.world/secure/api/v2.0/platformFiles/2501{?projection}',
   'templated': True},
  'download': {'href': 'https://vantage.earthi.world/secure/api/v2.0/platformFiles/2501/dl'},
  'owner': {'href': 'https://vantage.earthi.world/secure/api/v2.0/platformFiles/2501/owner'},
  'jobs': {'href': 'https://vantage.earthi.world/secure/api/v2.0/platformFiles/2501/jobs'},
  'collection': {'href': 'https://vantage.ea

#### 5.3 Get the download cost estimation

In [53]:
output_file='2501'
response = requests.get(endpoint + '/estimateCost/download/' + output_file, auth=(user,api))
response.status_code

200

In [54]:
response.json()

{'estimatedCost': 1, 'recurrence': 'ONE_OFF', 'currentWalletBalance': 112}

#### 5.4 Download the product

In [55]:
output_file='2501'
response = requests.get(endpoint + '/platformFiles/' + output_file + '/dl', auth=(user,api))
response.status_code

200

In [56]:
open('symmetricDifferenceOutput.zip', 'wb').write(response.content)

29157753